In [50]:
# Disable GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import numpy as np
import pandas as pd

# Load pretrained VGG16 model
model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)

# Dict to store layer weights
conv_layer_weights = {}

# Loop over layers and extract weights from convolutional layers only
for layer in model.layers:
    if 'conv' in layer.name:
        weights = layer.get_weights()
        if weights:  # weights[0] = kernel, weights[1] = bias
            kernel = weights[0].flatten()  # Flatten to 1D
            conv_layer_weights[layer.name] = kernel
# Find max length to align all columns
max_len = max(len(w) for w in conv_layer_weights.values())

# Pad all arrays to same length
for layer in conv_layer_weights:
    weights = conv_layer_weights[layer]
    padded = np.pad(weights, (0, max_len - len(weights)), constant_values=np.nan)
    conv_layer_weights[layer] = padded

# Convert to DataFrame (each column = conv layer)
df = pd.DataFrame(conv_layer_weights)

# Save to CSV
output_path = "/kaggle/working/vgg16_conv_weights_by_layer.csv"
df.to_csv(output_path, index=False)

print(f"✅ Saved weights layer-by-layer to: '/kaggle/working/vgg16_conv_weights_by_layer.csv'")

✅ Saved weights layer-by-layer to: '/kaggle/working/vgg16_conv_weights_by_layer.csv'


In [5]:
import pandas as pd
import numpy as np

# Load the CSV file (adjust path as needed)
csv_path = '/kaggle/working/vgg16_conv_weights_by_layer.csv'
df = pd.read_csv(csv_path)

# Check for any non-numeric or NaN values
df = df.apply(pd.to_numeric, errors='coerce')  # Coerce non-numeric values to NaN
df = df.fillna(0)  # Replace NaNs with 0 (or use another strategy if needed)

# Get global min and max
min_val = df.min().min()
max_val = df.max().max()

# Handle case where min == max to avoid division by zero
if min_val == max_val:
    scale = 1
else:
    scale = 255 / (max_val - min_val)

# Vectorized conversion function
def float_to_int8_vectorized(x):
    return np.round((x - min_val) * scale - 128).astype(np.int8)

# Apply the function using vectorized approach
df_int8 = df.apply(float_to_int8_vectorized)
# Save the result
output_path = '/kaggle/working/VGG_Int8_Weights.csv'
df_int8.to_csv(output_path, index=False)

print(f"✅ Conversion completed and saved to {output_path}")

✅ Conversion completed and saved to /kaggle/working/VGG_Int8_Weights.csv


In [8]:
pip install pillow numpy


Note: you may need to restart the kernel to use updated packages.


In [13]:
from PIL import Image
import numpy as np

# === INPUTS ===
image_path = '/kaggle/input/catyimage/cat.jpg'       # <-- change this to your image path
hex_output_path = '/kaggle/working/cat.hex'          # <-- output file path

# === PROCESS ===
# Load image and convert to grayscale
img = Image.open(image_path).convert('L')  # 'L' means grayscale

# Convert to NumPy array and flatten to 1D
pixels = np.array(img).flatten()

# Write each pixel as a hex value (2-digit) on its own line
with open(hex_output_path, 'w') as f:
    for px in pixels:
        f.write(f'{px:02X}\n')  # 2-digit uppercase hex

print(f'Image converted to hex and saved at: {hex_output_path}')


Image converted to hex and saved at: /kaggle/working/cat.hex


In [14]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'  # <-- update with your actual CSV path
hex_file_path = '/kaggle/working/cat.hex'            # <-- hex image file
output_path = '/kaggle/working/multiplied_output.txt'  # <-- output path

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 0].values

# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")

# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === SAVE RESULT ===
np.savetxt(output_path, result, fmt='%.4f')  # You can change formatting here

print(f"[SUCCESS] Result saved to {output_path}")


[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Result saved to /kaggle/working/multiplied_output.txt


In [20]:
import numpy as np
from PIL import Image

# Step 1: Load weights from .txt file
with open('/kaggle/working/multiplied_output.txt', 'r') as f:
    weights = np.array([float(line.strip()) for line in f])

# Step 2: Normalize to [0, 255]
min_val, max_val = weights.min(), weights.max()
normalized = 255 * (weights - min_val) / (max_val - min_val)
normalized = normalized.astype(np.uint8)

# Step 3: Reshape — Choose shape, e.g., 224*224
desired_shape = (224, 224)
if normalized.size != np.prod(desired_shape):
    raise ValueError(f"Size mismatch: can't reshape {normalized.size} elements to {desired_shape}")
image_array = normalized.reshape(desired_shape)

# Step 4: Convert to Image and Save
img = Image.fromarray(image_array, mode='L')  # 'L' = grayscale
img.save('weights_visualized.png')


In [21]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/cat.hex'
output_path = '/kaggle/working/multiplied_output.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 0].values

# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")

# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")


[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output.hex


In [26]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output.hex'
output_path = '/kaggle/working/multiplied_output1.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 1].values

# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")

# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")


[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output1.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output1.hex


In [32]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output1.hex'
output_path = '/kaggle/working/multiplied_output2.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 2].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")

# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")
        

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output2.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output2.hex


In [38]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output2.hex'
output_path = '/kaggle/working/multiplied_output3.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 3].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")
            

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output3.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output3.hex


In [39]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output3.hex'
output_path = '/kaggle/working/multiplied_output4.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 4].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output4.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output4.hex


In [40]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output4.hex'
output_path = '/kaggle/working/multiplied_output5.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 5].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output5.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output5.hex


In [41]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output5.hex'
output_path = '/kaggle/working/multiplied_output6.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 6].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output6.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output6.hex


In [42]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output6.hex'
output_path = '/kaggle/working/multiplied_output7.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 7].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output7.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output7.hex


In [43]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output7.hex'
output_path = '/kaggle/working/multiplied_output8.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 8].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output8.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output8.hex


In [44]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output8.hex'
output_path = '/kaggle/working/multiplied_output9.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 9].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output9.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output9.hex


In [45]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output9.hex'
output_path = '/kaggle/working/multiplied_output10.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 10].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output10.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output10.hex


In [46]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output10.hex'
output_path = '/kaggle/working/multiplied_output11.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 11].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output11.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output11.hex


In [47]:
import numpy as np
import pandas as pd

# === INPUT PATHS ===
weights_csv_path = '/kaggle/working/VGG_Int8_Weights.csv'
hex_file_path = '/kaggle/working/multiplied_output11.hex'
output_path = '/kaggle/working/multiplied_output12.hex'  # Output .hex file

# === LOAD HEX PIXELS ===
with open(hex_file_path, 'r') as f:
    hex_pixels = [int(line.strip(), 16) for line in f if line.strip() != '']

# === LOAD WEIGHTS CSV ===
weights_df = pd.read_csv(weights_csv_path)

# Extract first column (assumed to be first layer's weights)
first_layer_weights = weights_df.iloc[:, 12].values
# === VALIDATE SIZES ===
min_len = min(len(hex_pixels), len(first_layer_weights))
if len(hex_pixels) != len(first_layer_weights):
    print(f"[INFO] Mismatch in lengths — truncating to smallest length ({min_len})")
# === MULTIPLY ===
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])
# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex
# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')
print(f"[SUCCESS] Hex output saved to {output_path}")
result = np.multiply(hex_pixels[:min_len], first_layer_weights[:min_len])

# === CONVERT TO HEX STRINGS ===
# Round to nearest int before converting
result_int = np.round(result).astype(int)
hex_lines = [format(val & 0xFFFFFFFF, '08X') for val in result_int]  # 8-digit uppercase hex

# === SAVE TO .hex FILE ===
with open(output_path, 'w') as f:
    for line in hex_lines:
        f.write(line + '\n')

print(f"[SUCCESS] Hex output saved to {output_path}")

[INFO] Mismatch in lengths — truncating to smallest length (50176)
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output12.hex
[SUCCESS] Hex output saved to /kaggle/working/multiplied_output12.hex
